In [ ]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
ds = load_dataset("jahjinx/IMDb_movie_reviews")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

IMDB_train.csv:   0%|          | 0.00/47.6M [00:00<?, ?B/s]

IMDB_validation.csv:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

IMDB_test.csv:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})

In [ ]:
import pandas as pd
train_df = pd.DataFrame(ds['train'])
val_df = pd.DataFrame(ds['validation'])
test_df = pd.DataFrame(ds['test'])

In [ ]:
# remove duplicates
train_df.drop_duplicates(inplace=True)
val_df.drop_duplicates(inplace=True)
test_df.drop_duplicates(inplace=True)

In [ ]:
print(f"Shape of training data: {train_df.shape}")
print(f"Shape of validation data: {val_df.shape}")
print(f"Shape of testing data: {test_df.shape}")

Shape of training data: (35767, 2)
Shape of validation data: (3999, 2)
Shape of testing data: (9981, 2)


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove URLs
    comment = re.sub(r'https?://\S+|www\.\S+', '', comment)

    # Remove HTML tags
    comment = re.sub(r'<.*?>', '', comment)

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters and punctuation
    comment = re.sub(r'[^\w\s]', '', comment)


    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
train_df['text'] = train_df['text'].apply(preprocess_comment)
val_df['text'] = val_df['text'].apply(preprocess_comment)
test_df['text'] = test_df['text'].apply(preprocess_comment)

In [ ]:
%pip install mlflow dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.2/258.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.5/637.5 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import dagshub
dagshub.init(repo_owner='abhishekramgarh13', repo_name='sentiment-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=df5822a5-7cc3-4ce5-a3cb-26964860b638&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=d347b751869500f044a6f4fcee71f856f94f2627f77310041e59723845abbf87




Accessing as abhishekramgarh13

Initialized MLflow to track repo "abhishekramgarh13/sentiment-analysis"

Repository abhishekramgarh13/sentiment-analysis initialized!

In [ ]:
import mlflow

# setting tracking server
mlflow.set_tracking_uri('https://dagshub.com/abhishekramgarh13/sentiment-analysis.mlflow')

In [ ]:
mlflow.set_experiment("Model-Selection")

2025/01/28 01:28:24 INFO mlflow.tracking.fluent: Experiment with name 'Model-Selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/eb66d38530584046ad0eae80a7e64ef9', creation_time=1738027704054, experiment_id='1', last_update_time=1738027704054, lifecycle_stage='active', name='Model-Selection', tags={}>

In [ ]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 19.2 MB/s eta 0:00:00


In [ ]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
tfidf = TfidfVectorizer(max_features=2000)
X_train = tfidf.fit_transform(train_df['text'])
X_test = tfidf.transform(test_df['text'])
X_val = tfidf.transform(val_df['text'])
y_train = train_df['label']
y_test = test_df['label']
y_val = val_df['label']

In [ ]:
#Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, X_val ,y_train, y_test,y_val):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)
        y_pred_val = model.predict(X_val)



        # Log accuracy
        test_accuracy = accuracy_score(y_test, y_pred_test)
        mlflow.log_metric("test_accuracy", test_accuracy)
        val_accuracy = accuracy_score(y_val, y_pred_val)
        mlflow.log_metric("val_accuracy", val_accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred_test, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [ ]:
# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_val, model.fit(X_train, y_train).predict(X_val))

In [ ]:
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=20)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train, X_test,X_val, y_train, y_test,y_val)

# Run the experiment for LightGBM
run_optuna_experiment()

[I 2025-01-28 01:44:02,772] A new study created in memory with name: no-name-6b84cfd5-7302-4584-b7fd-92824fd85217
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 17982, number of negative: 17785
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.056515 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 383230
[LightGBM] [Info] Number of data points in the train set: 35767, number of used features: 2000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502754 -> initscore=0.011016
[LightGBM] [Info] Start training from score 0.011016


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-01-28 01:44:28,151] Trial 0 finished with value: 0.7251812953238309 and parameters: {'n_estimators': 99, 'learning_rate': 0.0017543333834701843, 'max_depth': 9}. Best is trial 0 with value: 0.7251812953238309.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 17982, number of negative: 17785
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.014988 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 383230
[LightGBM] [Info] Number of data points in the train set: 35767, number of used features: 2000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502754 -> initscore=0.011016
[LightGBM] [Info] Start training from score 0.011016


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-01-28 01:45:00,271] Trial 1 finished with value: 0.7699424856214053 and parameters: {'n_estimators': 124, 'learning_rate': 0.011622598018939919, 'max_depth': 10}. Best is trial 1 with value: 0.7699424856214053.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Number of positive: 17982, number of negative: 17785
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.628510 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 383230
[LightGBM] [Info] Number of data points in the train set: 35767, number of used features: 2000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502754 -> initscore=0.011016
[LightGBM] [Info] Start training from score 0.011016


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-01-28 01:56:35,343] Trial 19 finished with value: 0.8114528632158039 and parameters: {'n_estimators': 169, 'learning_rate': 0.027249755064840498, 'max_depth': 9}. Best is trial 11 with value: 0.8607151787946987.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 17982, number of negative: 17785
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.144403 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 383230
[LightGBM] [Info] Number of data points in the train set: 35767, number of used features: 2000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502754 -> initscore=0.011016
[LightGBM] [Info] Start training from score 0.011016


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025/01/28 01:57:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM at: https://dagshub.com/abhishekramgarh13/sentiment-analysis.mlflow/#/experiments/1/runs/931ad6995dc0436590162f58f0ca2c0c
🧪 View experiment at: https://dagshub.com/abhishekramgarh13/sentiment-analysis.mlflow/#/experiments/1
